##**Fake News: LSTM Analysis**

Using a dataset containing tweets and their classification as fake news or true news, in this notebook the LSTM is going to be implemented and illustrated

**What is done in the Notebook:**

· Load the required dependencies

· Define helper functions

· Load BERT from the Tensorflow Hub

· Load CSV files containing data

· Tokenization

· Build LSTM Model

· Save the best model and early stopping

· Fit the model

· Evaluate model results with test data

· Extract False Positives and False Negatives

**Load the required dependencies**

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py


In [ ]:
!pip install sentencepiece


     |████████████████████████████████| 1.2MB 6.7MB/s 


In [ ]:
pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.7/dist-packages (2.4.3)


In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
import keras
import tensorflow as tf
from keras.layers import LSTM
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

**Load CSV files containing data**

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cod_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fake-news/train")


In [ ]:
cod_train2, test= train_test_split(cod_train, random_state=0, test_size=0.2)
train, val =  train_test_split(cod_train2, random_state = 0,test_size=0.2)

In [ ]:
train['text'].head()

5043     RIO DE JANEIRO  —   Brazil’s president is faci...
6612     Black Activist Blasts Hillary Clinton Over Wik...
5131     (Table 9) Prec Metals= Mostly Silver \nJP Morg...
18362    Islamists around the Arab world celebrated Tur...
9430     14 Shares\n12 0 0 2 (Taksim Square - Gezi Park...
Name: text, dtype: object

**Tokenization**

In [ ]:
max_features = 100000 # max num words
maxlen = 250 
embedding_size = 200

# create the tokenizer with the maximum number of words to keep, 
# based on word frequency. 
# Only the most common num_words-1 words will be kept.
tokenizer = Tokenizer(num_words=max_features, oov_token = True)

train['text']=train['text'].astype(str)
test['text']=test['text'].astype(str)
val['text']=val['text'].astype(str)

# fit the tokenizer on the headlines
tokenizer.fit_on_texts(list(train['text']))

# Transforms each text in texts to a sequence of integers.
train_X = tokenizer.texts_to_sequences(train['text'])
test_X = tokenizer.texts_to_sequences(test['text'])
val_X = tokenizer.texts_to_sequences(val['text'])

# transforms a list of num_samples sequences (lists of integers)
# into a 2D Numpy array of shape (num_samples, num_timesteps).
train_X = pad_sequences(train_X, maxlen = maxlen)
test_X = pad_sequences(test_X, maxlen = maxlen)
val_X = pad_sequences(val_X, maxlen = maxlen)

train_y = train['label']
test_y = test['label']
val_y = val['label']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe


Load glove embedding set

In [ ]:

# load embeddings
EMBEDDING_FILE = '/content/drive/MyDrive/Colab Notebooks/fake-news/glove.6B.200d.txt'

def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

# first, build index mapping words in the embeddings set
# to their embedding vector
embeddings_index = {}
with open(EMBEDDING_FILE, encoding="utf8") as f:
    for line in f:
        word, coefs = get_coefs(*line.split(" "))
        embeddings_index[word] = coefs
            
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

# Random embedding vector for unknown words.
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_size))
# prepare embedding matrix
for word, i in word_index.items():
    if i >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        # words not found in embedding index will be random
        embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):



**Build BiLSTM Model**

- Activation Function: ReLU has been used as the activation function. It is a non-linear activation function, helping complex relationships in the data to be captured by the model.

- Optimizer: Adam optimizer, an adaptive learning rate optimizer.

- Loss function: The network will be trained to output a probability over the 2 classes using Sigmoid Loss.

In [ ]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, weights = [embedding_matrix]))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.TruePositives()])


**Save the best model and early stopping**

In [ ]:

# Save the model after every epoch.
saveBestModel = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/TFMColab/best_model.hdf5', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# Stop training when a monitored quantity has stopped improving.
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

**Fit the model**

In [ ]:
batch_size = 100
epochs = 25
model.fit(train_X, train_y, batch_size=batch_size, epochs=epochs, validation_data=(val_X, val_y), callbacks=[saveBestModel, earlyStopping])

Epoch 1/25
134/134 [==============================] - 293s 2s/step - loss: 0.6072 - accuracy: 0.6478 - precision: 0.6598 - recall: 0.6070 - true_positives: 2181.3852 - val_loss: 0.2484 - val_accuracy: 0.8960 - val_precision: 0.9342 - val_recall: 0.8523 - val_true_positives: 1420.0000
Epoch 2/25
134/134 [==============================] - 292s 2s/step - loss: 0.2211 - accuracy: 0.9197 - precision: 0.9149 - recall: 0.9268 - true_positives: 3156.1556 - val_loss: 0.1435 - val_accuracy: 0.9390 - val_precision: 0.9280 - val_recall: 0.9520 - val_true_positives: 1586.0000
Epoch 3/25
134/134 [==============================] - 288s 2s/step - loss: 0.1056 - accuracy: 0.9709 - precision: 0.9723 - recall: 0.9694 - true_positives: 3275.7481 - val_loss: 0.1433 - val_accuracy: 0.9450 - val_precision: 0.9661 - val_recall: 0.9226 - val_true_positives: 1537.0000
Epoch 4/25
134/134 [==============================] - 294s 2s/step - loss: 0.0453 - accuracy: 0.9859 - precision: 0.9874 - recall: 0.9844 - true_

**Evaluate model results with test data**


In [ ]:
model.metrics_names


['loss', 'accuracy', 'precision', 'recall', 'true_positives']

In [ ]:
loss, accuracy, precision, recall, true_positives = model.evaluate(test_X, test_y, batch_size=batch_size)


42/42 [==============================] - 23s 558ms/step - loss: 0.2124 - accuracy: 0.9524 - precision: 0.9407 - recall: 0.9674 - true_positives: 2045.0000


In [ ]:
mult_pr=precision*recall
sum_pr=precision+recall
div=mult_pr/sum_pr
f1_score=2*div

**Loss, Accuracy, Precision, Recall and F1**


In [ ]:
print('Loss:',loss)
print('Accuracy:',accuracy)
print('Precision:',precision)
print('Recall:',recall)
print('f1 score:',f1_score)
print('True positives:',true_positives)

Loss: 0.2124175727367401
Accuracy: 0.9524038434028625
Precision: 0.9406623840332031
Recall: 0.9673604369163513
f1 score: 0.9538246239175293
True positives: 2045.0


**Extract False Positives and False Negatives**

In [ ]:
pred_y = model.predict_classes(test_X, batch_size=batch_size)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
confusion_matrix(test_y, pred_y)


array([[1917,  129],
       [  69, 2045]])

In [ ]:
def getFP_FN_TP_lists(test_X, test_y, pred_y):
    FP_text = []
    FP_index = []
    FN_text = []
    FN_index = []
    TP_text = []
    TP_index = []
    for i in range(len(test_y)):
        if(pred_y[i]==1 and test_y[test_y.index[i]]==0):
            FP_text.append(test['text'][test_y.index[i]])
            FP_index.append(test_y.index[i])
        elif(pred_y[i]==0 and test_y[test_y.index[i]]==1):
            FN_text.append(test['text'][test_y.index[i]])
            FN_index.append(test_y.index[i])
        elif(pred_y[i]==1 and test_y[test_y.index[i]]==1):
            TP_text.append(test['text'][test_y.index[i]])
            TP_index.append(test_y.index[i])        
            
    return FP_text,FP_index,FN_text,FN_index,TP_text,TP_index

def getFP_FN_TP(test_X, test_y, pred_y):
    FP_text,FP_index,FN_text,FN_index,TP_text,TP_index = getFP_FN_TP_lists(test_X, test_y, pred_y)
    d_FP = {'FP_text':FP_text,'FP_index':FP_index}
    df_FP = pd.DataFrame(d_FP)
    d_FN = {'FN_text':FN_text,'FN_index':FN_index}
    df_FN = pd.DataFrame(d_FN)
    d_TP =  {'TP_text':TP_text,'TP_index':TP_index}
    df_TP = pd.DataFrame(d_TP)
    
    return df_FP,df_FN,df_TP

df_FP,df_FN, df_TP = getFP_FN_TP(test_X, test_y, pred_y)
df_FP.to_csv('FP_BiLSTMGlove.csv', index=True)
df_FN.to_csv('FN_BiLSTMGlove.csv', index=True)
df_TP.to_csv('TP_BiLSTMGlove.csv', index=True)